In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 102.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 18.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 79.4 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('diffGATGraphModel-movielens')
experiment.add_tags(['movielens', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/527798220a1140c5b8dfc2d8f2ce1335

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 8e-5,
    'train_batch_size': 4096,
    'train_print_every': 10,  
    'train_test_every': 10,
    'test_topk': 10,
    'test_batch_size': 2048
}

In [6]:
import os
os.listdir('/kaggle/input/data/leave-n-out/mvln')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/mvln/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['timestamp'], unit='s')
print(train.head())

   user_id  movie_id  rating  timestamp                date
0        1      3186       4  978300019 2000-12-31 22:00:19
1        1      1270       5  978300055 2000-12-31 22:00:55
2        1      1721       4  978300055 2000-12-31 22:00:55
3        1      1022       5  978300055 2000-12-31 22:00:55
4        1      2340       3  978300103 2000-12-31 22:01:43


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 211802
Количество explicit негативного фидбека 153484


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 327987
Количество implicit негативного фидбека 246536


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','movie_id','target','date']]
train.head()

,user_id,movie_id,target,date
0,1,3186,implicit_positive,2000-12-31 22:00:19
1,1,1270,explicit_positive,2000-12-31 22:00:55
2,1,1721,implicit_positive,2000-12-31 22:00:55
3,1,1022,explicit_positive,2000-12-31 22:00:55
4,1,2340,implicit_negative,2000-12-31 22:01:43


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)
train.columns = ['user_id', 'item_id', 'target', 'date']

In [14]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['timestamp'], unit='s')
print(test.head())

   user_id  movie_id  rating  timestamp                date
0        1      2687       3  978824268 2001-01-06 23:37:48
1        1       745       3  978824268 2001-01-06 23:37:48
2        1       588       4  978824268 2001-01-06 23:37:48
3        1         1       5  978824268 2001-01-06 23:37:48
4        1      2355       5  978824291 2001-01-06 23:38:11


In [15]:
test = test[['user_id','movie_id', 'date']]
test.columns = ['user_id', 'item_id', 'date']
test.head()

,user_id,item_id,date
0,1,2687,2001-01-06 23:37:48
1,1,745,2001-01-06 23:37:48
2,1,588,2001-01-06 23:37:48
3,1,1,2001-01-06 23:37:48
4,1,2355,2001-01-06 23:38:11


# MVP model v2

In [16]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(60394, 3)

In [17]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [18]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных item_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных item_id 3700
Количество уникальных user_id 6040


In [19]:
def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['item_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    return data

In [20]:
class SignedHeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        # print(h1.keys())
        # Собираем позитивный и негативный вклад для user
        pos_msgs = torch.zeros_like(h1['user'])
        neg_msgs = torch.zeros_like(h1['user'])
        for ft in self.pos_types:
            pos_msgs += h1[ft]  
        for ft in self.neg_types:
            neg_msgs += h1[ft]

        # signed-aggregation: attraction от pos, repulsion от neg
        user_h1 = pos_msgs - neg_msgs
        user_h1 = F.leaky_relu(self.norm1['user'](user_h1))
        user_h1 = self.dropout(user_h1)

        h1['user'] = user_h1

        h2 = self.conv2(h1, data.edge_index_dict)
        # print(h2.keys())
        pos_msgs2, neg_msgs2 = 0, 0
        for ft in self.pos_types:
            pos_msgs2 += h2[ft]
        for ft in self.neg_types:
            neg_msgs2 += h2[ft]

        user_h2 = pos_msgs2 - neg_msgs2
        user_h2 = F.leaky_relu(self.norm2['user'](user_h2))

        return user_h2

In [21]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[6040] },
  item={ node_id=[3700] },
  implicit_positive={ node_id=[6040] },
  explicit_positive={ node_id=[6040] },
  implicit_negative={ node_id=[6040] },
  expliсit_negative={ node_id=[6040] },
  (item, to_feedback_implicit_positive, implicit_positive)={ edge_index=[2, 327987] },
  (implicit_positive, feedback_to_item_implicit_positive, item)={ edge_index=[2, 327987] },
  (implicit_positive, to_user_implicit_positive, user)={ edge_index=[2, 6040] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 211802] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 211802] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 6040] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 246536] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 246536] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 604

In [22]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(3700, 0, 3699)

In [23]:
num_users = len(train['user_id'].unique())
num_items = train['item_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = SignedHeteroGNN(num_users, num_items, feedback_types)

In [24]:
model

SignedHeteroGNN(
  (user_emb): Embedding(6040, 32)
  (item_emb): Embedding(3700, 32)
  (fb_emb): ModuleDict(
    (implicit_positive): Embedding(6040, 32)
    (explicit_positive): Embedding(6040, 32)
    (implicit_negative): Embedding(6040, 32)
    (expliсit_negative): Embedding(6040, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (

In [25]:
model(data)

tensor([[ 1.8883e+00,  2.1260e+00, -5.6116e-03,  ...,  2.5069e-01,
          1.7280e-01, -7.0265e-03],
        [ 2.0908e+00,  1.8636e+00, -1.0640e-03,  ..., -3.5415e-03,
          8.6922e-01, -6.3107e-03],
        [ 1.8606e+00,  2.2887e+00, -1.9882e-03,  ..., -4.7144e-03,
          1.1525e+00, -7.1537e-03],
        ...,
        [ 1.0849e+00,  1.0376e+00,  4.5011e-01,  ..., -2.7861e-03,
          6.9082e-02, -2.2795e-03],
        [ 1.7740e+00,  2.1240e+00,  1.6758e-01,  ..., -4.9274e-03,
          5.5816e-01, -1.0044e-02],
        [ 1.8437e+00,  2.1750e+00, -1.9571e-03,  ...,  2.5669e-02,
          1.0347e+00, -7.2765e-03]], grad_fn=<LeakyReluBackward0>)

In [26]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [27]:
def evaluate(model, train_data,
             test_batch_size, top_k,
             viewed_items, interactions,
             device, test_step):
    """
    Оцениваем модель по всем пользователям:
    - строим топ-K рекомендации
    - фильтруем уже просмотренные
    - считаем recall@K, precision@K, map@K
    """
    model.eval()
    model.to(device)
    num_users = train_data['user'].node_id.shape[0]
    test_top_k = top_k * 150

    item_emb = model.item_emb.weight
    item_emb_t = item_emb.t().detach()
    del item_emb
    gc.collect()

    all_scores = []
    with torch.no_grad():
        for i in range(0, num_users, test_batch_size):
            end = min(i + test_batch_size, num_users)
            batch_users = torch.arange(i, end).to(device)
            user_e = model(
                data=train_data.to(device)
            )
            rating = torch.mm(user_e[batch_users].detach(), item_emb_t)
            _, topk = torch.topk(rating, k=test_top_k, dim=1)
            all_scores.append(topk)

            del user_e, rating
            gc.collect()
    all_scores = torch.cat(all_scores, dim=0).cpu().numpy()

    users_list, items, ranks = [], [], []
    for u in range(num_users):
        seen = viewed_items.get(u, set())
        recs = all_scores[u]
        mask = ~np.isin(recs, list(seen))
        filtered = recs[mask][:top_k]
        for rank, it in enumerate(filtered, 1):
            users_list.append(u)
            items.append(int(it))
            ranks.append(rank)
    reco_df = pd.DataFrame({
        'user_id': users_list,
        'item_id': items,
        'rank': ranks
    })

    metrics = {
        f'map@{top_k}': MAP(k=top_k),
        f'precision@{top_k}': Precision(k=top_k),
        f'recall@{top_k}': Recall(k=top_k),
        f'ndcg@{top_k}': NDCG(k=top_k)
    }
    results = calc_metrics(metrics=metrics,
                           reco=reco_df,
                           interactions=interactions)
    print(f"Step {test_step} — Test metrics:")
    for name, val in results.items():
        print(f"  {name}: {val:.9f}")
        experiment.log_metric(f"Test {name} vs step", val, step=test_step)
    del all_scores
    gc.collect()

    model.to(device)
    train_data.to(device)
    model.train()
    return results

In [28]:
def train_model(model,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048):
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 0
    print(f"Num of training examples: {num_train}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                evaluate(model, train_data,
                         test_batch_size, top_k,
                         viewed_items, interactions,
                         device, test_step=total_steps)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [29]:
experiment.log_parameters(hyperparameters)

In [30]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [31]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 211802
Epoch 1, Step 1, LR: 0.000080, Current Loss: 2.6263, Avg Loss: 2.6263
Diff stats — min: -23.1286, max: 19.9993, mean: -0.2984, std: 5.9134

Step 0 — Test metrics:
  precision@10: 0.004668874
  recall@10: 0.004668874
  ndcg@10: 0.005354564
  map@10: 0.001778638
Epoch 1, Step 10, LR: 0.000080, Current Loss: 2.4733, Avg Loss: 2.5438
Diff stats — min: -20.2335, max: 20.1637, mean: -0.1932, std: 5.6803

Step 9 — Test metrics:
  precision@10: 0.004983444
  recall@10: 0.004983444
  ndcg@10: 0.005696341
  map@10: 0.001884553
Epoch 1, Step 20, LR: 0.000080, Current Loss: 2.3095, Avg Loss: 2.4559
Diff stats — min: -22.8362, max: 20.9054, mean: -0.1682, std: 5.3234

Step 19 — Test metrics:
  precision@10: 0.005182119
  recall@10: 0.005182119
  ndcg@10: 0.005974567
  map@10: 0.001987057
Epoch 1, Step 30, LR: 0.000080, Current Loss: 2.1287, Avg Loss: 2.3776
Diff stats — min: -17.9582, max: 18.0628, mean: -0.0276, std: 4.9972

Step 29 — Test metrics:
  precision@10: 

KeyboardInterrupt: 

In [32]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 211802
Epoch 1, Step 1, LR: 0.000080, Current Loss: 0.4331, Avg Loss: 0.4331
Diff stats — min: -5.4665, max: 7.1258, mean: 1.3849, std: 1.6552

Step 0 — Test metrics:
  precision@10: 0.023377483
  recall@10: 0.023379323
  ndcg@10: 0.024330058
  map@10: 0.007909873
Epoch 1, Step 10, LR: 0.000080, Current Loss: 0.4191, Avg Loss: 0.4189
Diff stats — min: -4.3928, max: 7.4443, mean: 1.4475, std: 1.6609

Step 9 — Test metrics:
  precision@10: 0.023476821
  recall@10: 0.023478661
  ndcg@10: 0.024375896
  map@10: 0.007915018
Epoch 1, Step 20, LR: 0.000080, Current Loss: 0.4127, Avg Loss: 0.4187
Diff stats — min: -4.3412, max: 7.4131, mean: 1.4724, std: 1.6670

Step 19 — Test metrics:
  precision@10: 0.023625828
  recall@10: 0.023629507
  ndcg@10: 0.024537407
  map@10: 0.007985986
Epoch 1, Step 30, LR: 0.000080, Current Loss: 0.4165, Avg Loss: 0.4190
Diff stats — min: -4.7664, max: 7.5080, mean: 1.4725, std: 1.6879

Step 29 — Test metrics:
  precision@10: 0.023774834


In [33]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [34]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [35]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [36]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : diffGATGraphModel-movielens
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/527798220a1140c5b8dfc2d8f2ce1335
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [1166] : (-0.2983660101890564, 2.9147939682006836)
COMET INFO:     Diff stats (std) vs step [1166]  : (0.8144240379333496, 5.913432598114014)
COMET INFO:     Test map@10 vs step [1166]       : (0.0017786384421318198, 0.013094419040611092)
COMET INFO:     Test ndcg@10 vs step [1166]      : (0.005354564317200823, 0.03636796341204262)
COMET INFO:     Test precision@10 vs step [1166] : (0.004668874172185431, 0.03586092715231788